### RatS Day3SD recording info

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects

sess = subjects.sd.ratSday3[0]
print(sess.recinfo)

### Set probe configuration
Lets create probe configuration for a 128-channel probe

In [ ]:
%matplotlib widget
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- cambridge probe -----------
for i in range(6):
    chans = channel_groups[i]

    if len(chans)==10:
        n_cont = 5
        y_shift = [0,7.5]
    if len(chans)==11:
        n_cont = [6,5]
        y_shift = [0,7.5]
    if len(chans)==9:
        n_cont = [5,4]
        y_shift = [0,7,5]

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=n_cont,
        xpitch=16.5,
        ypitch=15,
        y_shift_per_column=y_shift,
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

shanks = []
for i in range(6,14):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = Probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.filename = sess.filePrefix.with_suffix(".probegroup.npy")
prbgrp.save()
plot_probe(prbgrp)

## Create epochs for your experimental paradigm
A typical experiment involves multiple epochs such pre sleep, running on track and then another sleep epoch.

In [ ]:
sess.paradigm.to_dataframe()

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0, 10819, 14818, 14818, 48419],
        "stop": [10818, 14817, 48418, 32818, 52055],
        "label": ["pre", "maze", "post", "sd", "re-maze"],
    }
)

paradigm = Epoch(epochs=epochs)
paradigm.filename = sess.filePrefix.with_suffix(".paradigm.npy")
paradigm.save()

# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([1])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
plotting.plot_artifact_epochs(artifact_epochs, signal)

## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations

signal = sess.eegfile.get_signal()
ripple_epochs = oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix(".ripple.npy")
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

## Spindles

In [ ]:
from neuropy.analyses import oscillations

signal = sess.eegfile.get_signal()
spindles = oscillations.detect_spindle_epochs(
    signal, sess.probegroup, freq_band=(7, 15), thresh=(1, 5), mindur=1, maxdur=10
)
spindles.filename = sess.filePrefix.with_suffix(".spindle")
# spindles.save()

In [ ]:
sess.recinfo.write_epochs(spindles)

# Importing spiketrains from Phy

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/data/Clustering/sessions/RatS/Day3SD/spykcirc/RatS_Day3SD_2020-11-29_07-53-30.GUI"
)
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains = phy_data.spiketrains
peak_chans = phy_data.peak_channels
# waveforms = phy_data.peak_waveforms
waveforms = phy_data.waveforms
waveforms_amplitude = phy_data.waveforms_amplitude
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id < 4] = "pyr"
neuron_type[neuron_type_id == 6] = "mua"
neuron_type[neuron_type_id == 8] = "inter"


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    waveforms_amplitude=waveforms_amplitude,
    neuron_type=neuron_type,
    shank_ids=np.array(shank_id).astype(int),
    metadata={"cluster_path": cluster_path},
)

# neurons.filename = sess.filePrefix.with_suffix('.neurons')
# neurons.save(sess.filePrefix.with_suffix('.neurons'))

In [ ]:
from neuropy.core import Epoch

bins = np.linspace(0, sess.eegfile.duration, 6)
labels = np.arange(len(bins) - 1).astype("str")

epoch = Epoch.from_array(bins[:-1], bins[1:], labels)
phy_data.calculate_metrics(epoch)

In [ ]:
neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    waveforms_amplitude=waveforms_amplitude,
    neuron_type=neuron_type,
    shank_ids=np.array(shank_id).astype(int),
    metadata={"cluster_path": cluster_path, "stability_metrics": phy_data.metrics},
)

# neurons.filename = sess.filePrefix.with_suffix('.neurons')
neurons.save(sess.filePrefix.with_suffix(".neurons.iso"))

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

from neuropy.plotting import plot_raster


plt.plot(phy_data.peak_waveforms[0])
plot_raster(neurons,color='jet',add_vert_jitter=True)

# BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua = sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()

In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix(".pbe")
pbe.save()

### Position

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / "position"
opti_data = OptitrackIO(dirname=opti_folder)

In [ ]:
import pandas as pd
from datetime import datetime

# ------- maze align corection ---------
# Errors were estimated from TTL and they were pretty much spot-on even when aligned with theta/delta speed
t_error = [0, 0.692, 0, 0, 0.100]

# ---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    # sync_time = rec_datetime['sync_nframes'][i]/rec_datetime['sync_rate'][i]
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(
        t_error[i], unit="sec"
    )
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)

x, y, z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z, x, y))

position = Position(traces=traces, t_start=0, sampling_rate=opti_data.sampling_rate)
position.save(sess.filePrefix.with_suffix(".position"))

### Linearize position

In [ ]:
from neuropy.utils import position_util
from neuropy import plotting

maze = sess.paradigm["re-maze"].flatten()
maze_pos = sess.position.time_slice(maze[0], maze[1])
linear_pos = position_util.linearize_position(maze_pos)
linear_pos.save(sess.filePrefix.with_suffix(".remaze.linear"))


fig = plotting.Fig(2, 4, size=(8.5, 5))

ax = fig.subplot(fig.gs[0])
ax.plot(maze_pos.x, maze_pos.y)

ax = fig.subplot(fig.gs[0, 1:])
ax.plot(maze_pos.time, maze_pos.x, "r")
ax.plot(maze_pos.time, maze_pos.y, "g")
ax.plot(linear_pos.time, linear_pos.x, "k")